# 7.1 전체 구조 (CNN)
CNN의 네트워크 구조를 살펴보며 전체 틀을 이해해보자<BR>
CNN도 지금까지 본 신경망과 같이 레고 블록처럼 계층을 조합항 만들 수 있다. 다만 **합성곱 계층(convolutional layer)** 과 **풀링 계층(pooling layer)** 가 새롭게 등장한다. <br>
<BR>
지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다. 이를 **완전연결(fully-connected, 전결합)** 이라고 하며, 완전히 연결된 계층을 **Affine 계층** 이라는 이름으로 구현했다.

Affine 계층을 사용하면, 가령 층이 5개인 완전연결 신경망은 아래와 같이 구현할 수 있다.<Br><BR>
<img src="https://velog.velcdn.com/images/jw5150/post/404b1898-f129-45f2-a5c5-4905f1a74f3a/image.png" width=1200 height=250></img><br><br> 
위와 같이 완전연결 신경망은 Affine 계층 뒤에 활성화 함수를 갖는 ReLU 계층(혹은 Sigmoid 계층)이 이어진다. 그럼 CNN 구조는 어떨까?

<img src="https://velog.velcdn.com/images/jw5150/post/b1f040df-5031-4ea2-a3df-8792ef545406/image.png" width=1200 height=250></img><br><br> 
위 그림과 같이 CNN에서는 새로운 '합성곱 계층(Conv)'과 '풀링 계층(Pooling)'이 추가된다. CNN의 계층은 'Conv-ReLU-(Pooling)' 흐름으로 연결된다(풀링 계층 생략하기도 함).<BR>
지금까지 'Affine-ReLU' 연결이 'Conv-ReLU-(Pooling)'으로 바뀌었다 생각

위 그림에서 또 주목할 점은 출력에 가까운 층에서는 지금까지의 'Affine-ReLU' 구성을 사용할 수 있다는 것이다. 마지막 출력 계층에서는 'Affine-Softmax' 조합을 그대로 사용한다.

# 7.2 합성곱 계층
CNN 에서는 **패딩(padding), 스트라이드(stride)** 등 CNN 고유의 용어가 등장한다. 또 각 계층 사이에는 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.

## 7.2.1 완전연결 계층의 문제점
지금까지 완전연결 계층(Affine 계층)을 사용했다. 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있다. <bR>
<Br>
완전연결 계층의 문제점은 무엇일까? 바로 '데이터의 형상이 무시'된다는 사실이다. 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로, 가로, 채널(색상)로 구성된 3차원 데이터이다. 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야 한다. 지금까지 MNIST 데이터셋을 사용한 사례에서는 형상이 (1, 28, 28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine 계층에 입력했다. <BR><BR>
이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨있다. 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것이다. 그러나 완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 취급하여 형상에 담긴 정보를 살릴 수 없다.<Br><BR>
한편, 합성곱 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할(가능성이 있는) 것이다.<br><BR>
CNN에서는 합성곱 계층의 입출력 데이터를 **특징 맵(feature map)** 이라고도 한다. 합성곱 계층의 입력 데이터를 **입력 특징 맵(input feature map)** 출력 데이터를 **출력 특징 맵(output feature map)** 이라고 하는 식이다. 이 책에서는 '입출력 데이터'와 '특징 맵'을 같은 의미로 사용한다.

# 7.2.2 합성곱 연산
합성곱 계층에서의 **합성곱 연산** 을 처리한다. 합성곱 연산은 이미지 처리에서 말하는 **필터 연산** 에 해당한다. 

<img src="https://velog.velcdn.com/images/jw5150/post/9d011f29-cb94-4b00-9510-27e9399b53b6/image.png"></img><br><br> 

위 그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용한다. 입력 데이터는 세로, 가로 방향의 형상을 가졌고, 필터 역시 마찬가지이다. 데이터와 필터의 형상을 (높이 height, 너비 width) 로 표기하며, 위 예에서는 입력 (4, 4), 필터 (3, 3), 출력 (2, 2)가 된다. 필터를 **커널** 이라고도 한다.

<img src="https://velog.velcdn.com/images/jw5150/post/775209a0-9cf9-4157-b9bb-099cd6a4486e/image.png"></img><br><br> 

합성곱 연산은 윈도우(회색 영역)를 일정 간격(stride)으로 이동하며 원소끼리 곱하고 그 총합을 구한다(이를 단일 곱셈-누산 fused multiply-add, FMA라 한다). 그리고 그 결과를 해당 장소에 저장하고 이 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성된다.

완전연결 신경망에서는 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 그동안의 '가중치'에 해당한다. 그리고 CNN에서도 편향이 존재한다. 

<img src="https://velog.velcdn.com/images/jw5150/post/a5a7b944-c860-416b-9c36-7540b63ce379/image.png"></img><br><br> 

편향까지 포함하면 위 그림과 같은 흐름이 된다. 편향은 필터를 적용한 후의 데이터에 더해진다. 그리고 편향은 항상 하나(1 x 1) 만 존재한다. 그 하나의 값을 필터를 적용한 모든 원소에 더한다.

## 7.2.3 패딩
합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예컨대 0)으로 채우기도 한다. 이를 **패딩(padding)** 이라고 한다. 예를 들어 아래 그림은 (4, 4) 크기의 입력 데이터에 폭이 1인 패딩을 적용한 모습이다. 폭 1짜리 패딩이라 하면 입력 데이터 사방 1픽셀을 특정 값으로 채운다.

<img src="https://velog.velcdn.com/images/jw5150/post/e85f8ac6-35d6-4b78-8df9-4156162e7b06/image.png"></img><br><br> 

위 그림 같이 처음에 크기가 (4, 4)인 입력 데이터에 패딩이 추가되어 (6, 6)이 된다. 이 입력에 (3, 3) 크기의 필터를 걸면 (4, 4) 크기의 출력 데이터가 생성된다. 이 예에서는 패딩을 1로 설정했지만, 2나 3 등 원하는 정수로 설정할 수 있다. 

**NOTE_** 패딩은 주로 출력 크기를 조정할 목적으로 사용된다. 예를 들어 (4, 4) 입력 데이터에 (3, 3) 필터를 적용하면 출력은 (2, 2)가 되어, 입력보다 2만큼 줄어든다. 이는 합성곱 연산을 몇 번이나 되풀이하는 심층 신경망에서 문제가 될 수 있다. 합성곱 연산을 거칠 때마다 크기가 작아지면 어느 시점에서는 출력 크기가 1이 되어버린다. 더 이상은 합성곱 연산을 진행할 수 없게 된다. 이러한 사태를 막기 위해 패딩을 사용한다. 앞의 예에서는 패딩의 폭을 1로 설정하니 (4, 4) 입력에 대한 출력이 같은 크기인 (4, 4)로 유지되었다. 한 마디로 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달할 수 있다.

## 7.2.4 스트라이드
필터를 적용하는 위치의 간격을 **스트라이드(stride)** 라고 한다. 아래는 스트라이드를 2로 한 그림이다. 필터를 적용하는 윈도우가 두 칸씩 이동한다.

<img src="https://velog.velcdn.com/images/jw5150/post/473e3a2d-cbe3-4f3c-b73a-30dc488fb7b9/image.png"></img><br><br> 

위 그림에서는 크기가 (7, 7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용한다. 이처럼 스트라이드는 필터를 적용하는 간격을 지정한다.

패딩이 커지면 출력의 크기가 커지고, 스트라이드가 커지면 출력의 크기가 작아진다. 이를 수식화하면 다음과 같다.

<img src="https://velog.velcdn.com/images/jw5150/post/78082213-e523-4aea-bb50-f537cf949977/image.png"></img><br><br> 

- 입력 크기: (H, W)
- 필터 크기: (FH, FW)
- 출력 크기: (OH, HW)
- 패딩: P
- 스트라이드: S

위 식은 정수로 나누어떨어져야 한다. 출력 크기가 정수가 아니면 오류를 내는 등의 대응을 해줘야한다. 덧붙여서, 딥러닝 프레임워크 중에는 값이 딱 나눠떨어지지 않을 때는 가장 가까운 정수로 반올림하는 등, 특별히 에러를 내지 않고 진행하도록 구현하는 경우도 있다.

## 7.2.5 3차원 데이터의 합성곱 연산

아래는 3차원 데이터의 합성곱 연산 예이다.

<img src="https://velog.velcdn.com/images/jw5150/post/f9164b15-c17a-4eb4-9768-cf1720b90452/image.png"></img><br><br> 

2차원일 때와 비교하면, 길이 방향(채널 방향)으로 특징 맵이 늘어났다. 채널 쪽으로 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는다.

<img src="https://velog.velcdn.com/images/jw5150/post/7455566b-73c5-4b18-bcb8-942d1b513a99/image.png"></img><br><br> 

주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것이다. 이 예에서는 모두 3개로 일치한다. 한편, 필터 자체의 크기는 원하는 값으로 설정할 수 있다(단 모든 채널의 필터가 같은 크기여야 한다). 이 예에서는 필터의 크기가 (3, 3)이지만 원한다면 바꿀 수 있다. 

## 7.2.6 블록으로 생각하기
3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽다. 

<img src="https://velog.velcdn.com/images/jw5150/post/5222c5b3-304d-4674-a9bc-7b4b2f13a22b/image.png"></img><br><br> 

이 예에서 출력 데이터는 한 장의 특징 맵이다. 한 장의 특징 맵을 다른 말로 하면 채널이 1개인 특징 맵이다. 그럼 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야할까?<bR>
필터(가중치)를 다수 사용하는 것이다.

<img src="https://velog.velcdn.com/images/jw5150/post/657705dc-132d-4c1c-92e3-360745912509/image.png"></img><br><br> 

필터를 FN개(필터의 개수) 적용하면 출력 맵도 FN개가 생성된다. 그리고 그 FN개의 맵을 모으면 형상이 (FN, OH, OW)인 블록이 완성된다. 이 완성된 블록을 다음 계층으로 넘기겠따는 것이 CNN의 처리 흐름이다.

합성곱 연산에서는 필터의 수도 고려해야 한다. 그런 이유로 필터의 가중치 데이터는 4차원 데이터이며 (출력 채널 수, 입력  채널 수, 높이, 너비) 순으로 쓴다. 

예를 들어, 채널 수가 3, 크기가 (5, 5)인 필터가 20개면 (20, 3, 5, 5)로 쓴다.

<img src="https://velog.velcdn.com/images/jw5150/post/79186c7a-425d-4466-b02f-6333a72f4554/image.png"></img><br><br> 

편향은 채널 하나에 값 하나씩으로 구성된다. 이 예에서는 편향의 현상은 (FN, 1, 1)이고, 필터의 출력 결과의 형상은 (FN, OH, OW)이다. 이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해진다.

## 7.2.7 배치 처리
신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했다. 완전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했다.<br>
<br>
합성곱 연산도 마찬가지로 배치 처리를 지원한다. 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다. 구체적으로는 데이터를 (데이터 수, 채널 수, 높이, 너비) 순으로 저장한다. 데이터가 N개일 때 위 그림을 배치 처리한다면 데이터 형태가 아래처럼 된다.

<img src="https://velog.velcdn.com/images/jw5150/post/ab374e7a-06d2-48b3-83df-dacdb0f04b6e/image.png"></img><br><br> 

배치 처리 시의 데이터 흐름을 나타낸 위 그림을 보면 각 데이터의 선두에 배치용 차원을 추가한다. 이처럼 데이터는 4차원 형상을 가진 채 각 계층을 타고 흐른다. 여기에서 주의할 점으로는 신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것이다. 즉, N회 분의 처리를 한 번에 수행한다.

## 7.3 풀링 계층
풀링은 세로, 가로 방향의 공간을 줄이는 연산이다. 예를 들어 아래와 같이 2 x 2 영역을 원소 하나로 집약하여 공간 크기를 줄인다.

<img src="https://velog.velcdn.com/images/jw5150/post/a4226fa1-03b8-4f9f-baf1-02dbabd6e7d1/image.png"></img><br><br> 

위 그림은 2 x 2 **최대 풀링(max pooling)** 을 스트라이드 2로 처리하는 순서이다. 최대 풀링은 최댓값(max)을 구하는 연산으로, '2 x 2'는 대상 영역의 크기를 뜻한다. 즉 2 x 2 최대 풀링은 그림과 같이 2 x 2 크기의 영역에서 가장 큰 원소를 하나를 꺼낸다. 또, 스트라이드는 이 예에서는 2로 설정했으므로 2 x 2 윈도우가 원소 2칸 간격으로 이동한다. 참고로, 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통이다. 예를 들어 윈도우가 3 x 3이면 스트라이드는 3, 4 x 4이면 스트라이드를 4로 설정한다.

**WARNING_** 풀링은 최대 풀링 외에도 평균 풀링 등이 있따. 최대 풀링은 대상 영역에서 최댓값을 취하는 연산인 반면, 평균 풀링은 대상 영역의 평균을 계산한다. 이미지 인식 분야에서는 주로 최대 풀링을 사용한다.

## 7.3.1 풀링 계층의 특징
1. 학습해야 할 매개변수가 없다<br><BR>
풀링 계층은 합성곱 계층과 달리 학습해야 할 매개변수가 없다. 풀링은 대상 영역에서 최댓값이나 평균을 취하는 명확한 처리이므로 특별히 학습할 것이 없다.<BR><BR>
2. 채널 수가 변하지 않는다<BR><BR>
풀링 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다. 아래처럼 채널마다 독립적으로 계산하기 때문이다.<BR><BR>
<img src="https://velog.velcdn.com/images/jw5150/post/c8c07f1d-d011-4ccf-9f6f-776941872b8a/image.png" width=800 height=400></img><br><br> 
3. 입력의 변화에 영향을 적게 받는다(강건하다)<BR><BR>
입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다. 예를 들어 아래에서 입력 데이터의 차이(데이터가 오른쪽으로 1칸씩 이동)를 풀링이 흡수해 사라지게 하는 모습을 보여준다.<Br><Br>
<img src="https://velog.velcdn.com/images/jw5150/post/efa2d4c0-1055-4af0-8e03-99dd5373b479/image.png" width=1000 height=250></img><br><br> 
입력 데이터가 가로로 1원소만큼 어긋나도 출력은 같다(데이터에 따라서는 다를 수도 있다)

## 7.4 합성곱/풀링 계층 구현하기

합성곱 계층과 풀링 계층을 파이썬으로 구현해보자

## 7.4.1 4차원 배열
앞에서 설명한 대로 CNN에서 계층 사이를 흐르는 데이터는 4차원이다. 예를 들어 데이터의 형상이 (10, 1, 28, 28)이라면, 이는 높이 28, 너비 28, 채널 1개인 데이터가 10개라는 이야기다.

In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

여기에서 (10개 중) 첫 번째 데이터에 접근하려면 단순히 x[0]이라고 쓴다. 마찬가지로 두 번째 데이터는 x[1] 위치에 있다.

In [2]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


또, 첫 번째 데이터의 첫 채널의 공간 데이터에 접근하려면 다음과 같이 쓴다.

In [7]:
x[0, 0].shape # 또는 x[0][0]

(28, 28)

## 7.4.2 im2col로 데이터 전개하기
합성곱 연산을 곧이곧대로 구현하려면 for 문을 겹겹이 써야한다. 이는 구현이 어렵고, 넘파이에 for 문을 사용하면 성능이 떨어진다는 단점이 있다(넘파이에서는 원소에 접근할 때 for 문을 사용하지 않는게 바람직하다). for 문 대신 **im2col** 이라는 편의 함수를 사용해 간단하게 구현해보자

im2col은 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수이다. 아래 그림과 같이 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀐다(정확히는 배치 안의 데이터 수까지 포함된 4차원 데이터를 2차원으로 변환한다).

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F5a4T2%2FbtqGvQi5TOO%2FO52OKATjFmFqnzNBKyJtvK%2Fimg.png" width=800 height=400></img><br><br> 

im2col은 필터링하기 좋게 입력 데이터를 전개한다. 구체적으로는 아래 그림과 같이 입력 데이터에서 필터를 적용하년 영역(3차원 블록)을 한 줄로 늘어놓는다. 이 전개를 필터를 적용하는 모든 영역에서 수행하는 게 im2col이다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FRIVwv%2FbtqGuGA8RZC%2Fy06FG9LVbjrOHEOE15X1YK%2Fimg.png" width=800 height=350></img><br><br> 

위 그림에서는 보기에 좋게끔 스트라이드를 크게 잡아 필터의 적용 영역이 겹치지 않도록 했지만, 실제 상황에서는 영역이 겹치는 경우가 대부분이다. 필터 적용 영역이 겹치게 되면 im2col로 전개한 후의 원소 수가 원래 블록의 원소 수보다 많아진다. 그래서 im2col을 사용해 구현하면 메모리를 더 많이 소비하는 단점이 있다. 하지만 컴퓨터는 큰 행렬을 묶어서 계산하는 데 탁월하다. 예를 들어 행렬 계산 라이브러리(선형 대수 라이브러리) 등은 행렬 계산에 고도로 최적화되어 큰 행렬의 곱셈을 빠르게 계산할 수 있다. 그래서 문제를 행렬 계산으로 만들면 선형 대수 라이브러리를 활용해 효율을 높일 수 있다.

**NOTE_** im2col은 'image to column', 즉 '이미지에서 행렬로'라는 뜻이다. 카페(Caffe)와 체이너(Chainer) 등의 딥러닝 프레임워크는 im2col이라는 이름의 함수를 만들어 합성곱 계층을 구현할 때 이용하고 있다.

im2col로 입력 데이터를 전개한 다음에는 합성곱 계층의 필터(가중치)를 1열로 전개하고, 두 행렬의 내적을 계산하면 된다.<BR><BR>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FMJz5T%2FbtqGAgALqaK%2FELMpSWG0f0odXIN2Fa7dp0%2Fimg.png" width=800 height=450></img><br><br> 
위 그림과 같이 im2col 방식으로 출력한 결과는 2차원 행렬이다. CNN은 데이터를 4차원 배열로 저장하므로 2차원인 출력 데이터를 4차원으로 변형(reshape)한다. 이상이 합성곱 계층의 구현 흐름이다.


## 7.4.3 합성곱 계층 구현하기
im2col 함수는 common/util.py 에 구현되어 있다.<Br>
im2col 함수의 인터페이스는 다음과 같다.

im2col(input_data, filter_h, filter_w, stride=1, pad=0)<br><BR>
- input_data: (데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이뤄진 입력 데이터
- filter_h: 필터의 높이
- filter_w: 필터의 너비
- stride: 스트라이드
- pad: 패딩

이 im2col은 '필터 크기', '스트라이드', '패딩'을 고려하여 입력 데이터를 2차원 배열로 전개한다.

In [8]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0) 
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

(9, 75)
(90, 75)


여기에서는 두 가지 예를 보여주고 있다. 첫 번째는 배치 크기가 1(데이터 1개), 채널은 3개, 높이-너비가 7x7의 데이터이고, 두 번째는 배치 크기만 10이고 나머지는 첫 번째와 같다. <br><BR>
im2col 함수를 적용한 두 경우 모두 2번째 차원의 원소는 75개이다. 이 값은 필터의 원소 수와 같다(채널 3개, 5x5 데이터). 또한, 배치 크기가 1일 때는 im2col의 결과의 크기가 (9, 75)이고, 10일 때는 그 10배인 (90, 75) 크기의 데이터가 저장된다.

이 im2col을 사용하여 합성곱 계층을 구현해보자

In [9]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)  ###
        col_W = self.W.reshape(FN, -1).T        ###
        out = np.dot(col, col_W) + self.b           ###

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

합성곱 계층은 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받아 초기화한다. 필터는 (FN, C, FH, FW)의 4차원 형상이다. 여기서 FN은 필터 개수, C는 채널, FH는 필터 높이, FW는 필터 너비이다.<Br><BR>

'###' 을 주석으로 달아놓은 부분에서 입력 데이터를 im2col로 전개하고 필터도 reshape을 사용해 2차원 배열로 전개한다. 그리고 이렇게 전개한 두 행렬의 내적을 구한다.

필터를 전개하는 부분('###')은 <Br><BR>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FMJz5T%2FbtqGAgALqaK%2FELMpSWG0f0odXIN2Fa7dp0%2Fimg.png" width=800 height=450></img><br><br> 
위 그림에서 보듯 각 필터 블록을 1줄로 펼쳐 세운다. 이때 reshape의 두 번째 인수를 -1로 지정했는데, 이는 reshape이 제공하는 기능이다. reshape에 -1을 지정하면 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 적절히 묶어준다. 앞의 코드에서 (10, 3, 5, 5) 형상을 한 다차원 배열 W의 원소 수는 총 750개이다. 이 배열에 reshape(10, -1)을 하면 (10, 75)인 배열로 만들어준다.

다음으로 forward 구현의 마지막에서는 출력 데이터를 적절한 형상으로 바꿔준다. 이때 넘파이의 transpose 함수를 사용하는데, 이는 다차원 배열의 축 순서를 바꿔주는 함수이다.

out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)<br><BR>
<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_transpose.png" width=800 height=250></img><br><br> 
위와 같이 인덱스(0부터 시작)를 지정하여 축의 순서를 변경한다.

합성곱 계층의 역전파에서는 im2col을 역으로 처리하는 col2im 함수를 사용하면 된다.

## 7.4.4 풀링 계층 구현하기
풀링 계층 구현도 합성곱 계층과 마찬가지로 im2col을 사용해 입력 데이터를 전개한다. 단, 풀링의 경우엔 채널 쪽이 독립적이라는 점이 합성곱 계층 때와 다르다. 구체적으로는 아래와 같이 풀링 적용 영역을 채널마다 독립적으로 전개해야한다.<Br><BR>

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_pooling_imp.png" width=800 height=650></img><br><br> 

일단 이렇게 전개한 후, 전개한 행렬에서 행별 최댓값을 구하고 출력 데이터의 크기 (N, C, OH, OW)에 맞게 변환(성형)한다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_pooling_flow.png" width=800 height=400></img><br><br> 

In [10]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 최댓값(2)
        out = np.max(col, axis=1)
        
        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

풀링 계층 구현은 위 그림과 같이 다음의 세 단계로 진행된다.<BR><BR>
1. 입력 데이터를 전개한다.
2. 행별 최댓값을 구한다.
3. 적절한 모양으로 변환(성형)한다.<Br><BR><BR>
**NOTE_** 최댓값 계산에는 넘파이의 np.max 메서드를 사용할 수 있다. np.max는 인수로 축(axis)을 지정할 수 있는데, 이 인수로 지정한 축마다 최댓값을 구할 수 있다. 가령 np.max(x, axis=1)과 같이 쓰면 입력 x의 1번째 차원의 축마다 최댓값을 구한다.
<br><BR>
2차원 배열, 즉 행렬이라면 axis=0은 열 방향, axis=1은 행 방향을 뜻한다.

# 7.5 CNN 구현하기
<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_imp.png" width=800 height=300></img><br><br> 

이 그림의 CNN 네트워크를 SimpleConvNet이라는 이름의 클래스로 구현하자<BR>
코드는 simple_convnet.py를 참고하자

우선 SimpleConvNet의 초기화를 살펴보자 <BR><BR>
**초기화 때 받는 인수**<BR>
- input_dim: 입력 데이터(채널 수, 높이, 너비)의 차원
- conv_param: 합성곱 계층의 하이퍼파라미터(딕셔너리). 딕셔너리의 키는 다음과 같다.<BR>
&emsp; - filter_num: 필터 수<BR>
&emsp; - filter_size: 필터 크기<BR>
&emsp; - stride: 스트라이드<BR>
&emsp; - pad: 패딩<BR>
&emsp; - hidden size: 은닉층(완전연결)의 뉴런 수<BR>
&emsp; - output size: 출력층(완전연결)의 뉴런 수<BR>
&emsp; - weight_init_std: 초기화 때의 가중치 표준편차<BR>

여기에서 합성곱 계층의 하이퍼파라미터는 딕셔너리 형태로 주어진다(conv_param).<BR>
이것은 필요한 하이퍼파라미터 값이 예컨대 {'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1} 처럼 저장된다는 뜻이다.<BR><BR>
SimpleConvNet의 초기화는 코드가 길어지므로 세 부분으로 나눠 설명하겠다.

In [16]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

여기에서는 초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼낸다(나중에 쓰기 쉽도록). 그리고 합성곱 계층의 출력 크기를 계산한다. 이어서 다음 코드는 가중치 매개변수를 초기화하는 부분이다.

In [18]:
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


학습에 필요한 매개변수는 1번째 층의 합성곱 계층과 나머지 두 완전연결 계층의 가중치와 편향이다. 이들 매개변수를 인스턴스 변수 params 딕셔너리에 저장한다. 1번째 층의 합성곱 계층의 가중치를 W1, 편향을 b1이라는 키로 저장한다. 마찬가지로 2번째 층의 완전연결 계층의 가중치와 편향을 W2와 b2, 마지막 3번째 층의 완전연결 계층의 가중치와 편향을 W3와 b3라는 키로 각각 저장한다.<BR><BR>
마지막으로 CNN을 구성하는 계층들을 생성하자

In [ ]:
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()


순서가 있는 딕셔너리(OrderedDict)인 layers에 계층들을 차례로 추가한다. 마지막 SoftmaxWithLoss 계층만큼은 lastLayer라는 별도 변수에 저장한다.

이상이 SimpleConvNet의 초기화이다. 이렇게 초기화를 마친 다음에는 추론을 수행하는 predict 메서드와 손실 함수의 값을 구하는 loss 메서드를 다음과 같이 구현할 수 있다.

In [ ]:
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

인수 x는 입력 데이터, t는 정답 레이블이다. 추론을 수행하는 predict 메서드는 초기화 때 layers에 추가한 계층을 맨 앞에서부터 차례로 forward 메서드를 호출하여 그 결과를 다음 계층에 전달한다. 손실 함수를 구하는 loss 메서드는 predict 메서드의 결과를 인수로 마지막 층의 forward 메서드를 호출한다. 즉, 첫 계층부터 마지막 계층까지 forward를 처리한다.<BR><BR>
이어서 오차역전파법으로 기울기를 구하는 구현은 다음과 같다.

In [ ]:
    def gradient(self, x, t):
        
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

매개변수의 기울기는 오차역전파법으로 구한다. 이 과정은 순전파와 역전파를 반복한다. 마지막으로 grads라는 딕셔너리 변수에 각 가중치 매개변수의 기울기를 저장한다.
<br><BR>
SimpleConvNet을 MNIST 데이터셋으로 학습하면훈련 데이터에 대한 정확도는 98.96% 가 된다.(train_convnet.py)

# 7.6 CNN 시각화하기
CNN을 구성하는 합성곱 계층은 입력으로 받은 이미지 데이터에서 '무엇을 보고 있는'걸까?
<BR>
이번 절에서는 합성곱 계층을 시각화해서 CNN이 보고 있는 것이 무엇인지 알아보자

## 7.6.1 1번째 층의 가중치 시각화하기
MNIST 데이터셋으로 진행한 간단한 CNN 학습에서 1번째 층의 필터를 이미지로 나타내 보면 다음과 같다.

<img src="https://velog.velcdn.com/images/jw5150/post/14de4595-84e2-4499-bc10-0e1963aa4ad6/image.png" width=800 height=300></img><br><br> 

학습 전 필터는 무작위로 초기화되고 있어 흑백의 정도에 규칙성이 없다. 한편, 학습을 마친 필터는 규칙성이 있는 이미지가 되었다. 흰색에서 검은색으로 점차 변화하는 필터와 덩어리(블롭 - blob)가 진 필터 등, 규칙을 띄는 필터로 바뀌었다.

위 그림의 오른쪽과 같이 규칙성 있는 필터는 '무엇을 보고 있는'걸까? 그것은 에지(색상이 바뀐 경계선)와 블롭(국소적으로 덩어리진 영역) 등을 보고있다. 가령 왼쪽 절반이 흰색이고 오른쪽 절반이 검은색인 필터는 아래 그림과 같이 세로 방향의 에지에 반응하는 필터이다.

<img src="https://velog.velcdn.com/images/jw5150/post/8d99cd8e-cc06-408a-a908-2e1eb3cd8a1b/image.png" width=1000 height=500></img><br><br> 

위 그림은 학습된 필터 2개를 선택하여 입력 이미지에 합성곱 처리를 한 결과로, '필터 1'은 세로 에지에 반응하며 '필터 2'는 가로 에지에 반응한다.<BR>
이처럼 합성곱 계층의 필터는 에지나 블롭 등의 원시적인 정보를 추출할 수 있다. 이런 원시적인 정보가 뒷단 계층에 전달된다는 것이 앞에서 구현한 CNN에서 일어나는 일이다.

## 7.6.2 층 깊이에 따른 추출 정보 변화
1번째 층의 합성곱 계층에서는 에지나 블롭 등의 저수준 정보가 추출된다 치고, 그럼 겹겹이 쌓인 CNN의 각 계층에서는 어떤 정보가 추출될까? 딥러닝 시각화에 관한 연구에 따르면, 계층이 깊어질수록 추출되는 정보(정확히는 강하게 반응하는 뉴런)는 더 추상화된다는 것을 알 수 있다.

아래 그림은 일반 사물 인식(자동차나 개 등)을 수행한 8층의 CNN이다. 이 네트워크 구조는 AlexNet이라 하는데, 합성곱 계층과 풀링 계층을 여러 겹 쌓고, 마지막으로 완전연결 계층을 거쳐 결과를 출력하는 구조이다. 아래 그림에서 블록으로 나타낸 것은 중간 데이터이며, 그 중간 데이터에 합성곱 연산을 연속해서 적용한다.

<img src="https://velog.velcdn.com/images/jw5150/post/71fd4ba3-9e16-4325-bafd-515486869126/image.png" width=1000 height=450></img><br><br> 

딥러닝의 흥미로운 점은 위 그림과 같이 합성곱 계층을 여러 겹 쌓으면, 층이 깊어지면서 더 복잡하고 추상화된 정보가 추출된다는 것이다. 처음 층은 단순한 에지에 반응하고, 이어서 텍스처에 반응하고, 더 복잡한 사물의 일부에 반응하도록 변화한다. 다시 말하면 사물의 '의미'를 이해하도록 변화하는 것이다.

# 7.7 대표적인 CNN
CNN의 원조인 LeNet과 딥러닝이 주목받도록 이끈 AlexNet을 소개한다.

## 7.7.1 LeNet
**LeNet** 은 손글씨 숫자를 인식하는 네트워크로, 아래와 같이 합성곱 계층과 풀링 계층(정확히는 단순히 '원소를 줄이기'만 하는 서브샘플링 계층)을 반복하고, 마지막으로 완전연결 계층을 거치면서 결과를 출력한다.

<img src="https://velog.velcdn.com/images/jw5150/post/1d825f06-f2c8-4023-b53c-907a4b5091e1/image.png" width=1000 height=320></img><br><br> 

LeNet과 '현재의 CNN'을 비교하면 몇 가지 면에서 차이가 있다. 첫 번째 차이는 활성화 함수이다. LeNet은 시그모이드 함수를 사용하는 데 반해, 현재는 주로 ReLU를 사용한다. 또 원래의 LeNet은 서브샘플링을 하여 중간 데이터의 크기가 작아지지만 현재는 최대 풀링이 주류이다.

## 7.7.2 AlexNet
<img src="https://velog.velcdn.com/images/jw5150/post/2fea1cc0-5311-4796-9ad7-32478e4ec7c8/image.png" width=1000 height=350></img><br><br> 

AlexNet은 합성곱 계층과 풀링 계층을 거듭하며 마지막으로 완전연결 계층을 거쳐 결과를 출력한다. <BR>
- 활성화 함수로 ReLU를 사용
- LRN(Local Response Normalization)이라는 국소적 정규화를 실시하는 계층을 이용
- 드롭아웃을 사용


LeNet과 AlexNet에 큰 차이는 없다. 그러나 이를 둘러싼 환경과 컴퓨터 기술이 큰 진보를 이뤘다. 대량의 데이터를 누구나 얻을 수 있게 되었고, 병렬 계산에 특화된 GPU가 보급되면서 대량의 연산을 고속으로 수행할 수 있게 되었다. 빅 데이터와 GPU, 이것이 딥러닝 발전의 큰 원동력이다.